# Broadway Tony Producer Analysis

**Research Question:** How does the number of producers (lead and co-producers) relate to:
1. Tony Award nominations and wins
2. Weekly Broadway grosses
3. Show run length (survival)

**Data:** Broadway shows from 2010-2011 season onward

**Methods:**
- Descriptive statistics
- Logistic regression (Tony wins)
- Count models (Tony nominations)
- Panel regression (weekly grosses)
- Cox proportional hazards (survival analysis)

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys

# Statistical modeling
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats

# Survival analysis
from lifelines import KaplanMeierFitter, CoxPHFitter
from lifelines.statistics import logrank_test

# Add parent directory to path
sys.path.append(str(Path.cwd().parent))
import config

# Plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Display options
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 3)

print("Imports successful!")

## 1. Load Data

In [ ]:
# Load master datasets
shows = pd.read_csv(config.SHOWS_CSV)
print(f"Loaded {len(shows)} shows")

# Load weekly grosses (if available)
if config.WEEKLY_GROSSES_CSV.exists():
    weekly_grosses = pd.read_csv(config.WEEKLY_GROSSES_CSV)
    weekly_grosses['week_ending_date'] = pd.to_datetime(weekly_grosses['week_ending_date'])
    print(f"Loaded {len(weekly_grosses)} weekly grosses records")
else:
    weekly_grosses = pd.DataFrame()
    print("WARNING: Weekly grosses data not available")

# Load survival panel (if available)
if config.SURVIVAL_PANEL_CSV.exists():
    survival_panel = pd.read_csv(config.SURVIVAL_PANEL_CSV)
    print(f"Loaded {len(survival_panel)} shows for survival analysis")
else:
    survival_panel = pd.DataFrame()
    print("WARNING: Survival panel not available")

In [ ]:
# Inspect shows data
shows.head(10)

In [ ]:
# Data summary
shows.info()

## 2. Descriptive Statistics

### Producer Counts

In [ ]:
# Producer count summary statistics
print("Producer Count Statistics")
print("=" * 60)
print(shows['producer_count_total'].describe())
print("\nLead Producer Count:")
print(shows['lead_producer_count'].describe())
print("\nCo-Producer Count:")
print(shows['co_producer_count'].describe())

In [ ]:
# Visualize producer count distribution
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].hist(shows['producer_count_total'].dropna(), bins=20, edgecolor='black')
axes[0].set_xlabel('Total Producer Count')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Total Producer Count')

axes[1].hist(shows['lead_producer_count'].dropna(), bins=15, edgecolor='black', color='steelblue')
axes[1].set_xlabel('Lead Producer Count')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Lead Producer Count')

axes[2].hist(shows['co_producer_count'].dropna(), bins=15, edgecolor='black', color='coral')
axes[2].set_xlabel('Co-Producer Count')
axes[2].set_ylabel('Frequency')
axes[2].set_title('Distribution of Co-Producer Count')

plt.tight_layout()
plt.savefig(config.OUTPUT_FIGURES_DIR / 'producer_count_distributions.png', dpi=300, bbox_inches='tight')
plt.show()

### Tony Outcomes

In [ ]:
# Tony outcomes summary
print("Tony Award Statistics")
print("=" * 60)
print(f"Shows with any Tony win: {shows['tony_win_any'].sum()} ({100*shows['tony_win_any'].mean():.1f}%)")
print(f"Shows with major Tony win: {shows['tony_major_win'].sum()} ({100*shows['tony_major_win'].mean():.1f}%)")
print(f"\nAverage nominations per show: {shows['tony_nom_count'].mean():.2f}")
print(f"Average wins per show: {shows['tony_win_count'].mean():.2f}")

In [ ]:
# Producer counts by Tony win status
print("\nProducer Counts by Tony Win Status")
print("=" * 60)
print("\nShows WITHOUT any Tony win:")
print(shows[shows['tony_win_any'] == 0]['producer_count_total'].describe())
print("\nShows WITH Tony win(s):")
print(shows[shows['tony_win_any'] == 1]['producer_count_total'].describe())

# T-test
no_win = shows[shows['tony_win_any'] == 0]['producer_count_total'].dropna()
has_win = shows[shows['tony_win_any'] == 1]['producer_count_total'].dropna()
t_stat, p_val = stats.ttest_ind(no_win, has_win)
print(f"\nT-test: t={t_stat:.3f}, p={p_val:.4f}")

In [ ]:
# Visualize producer count vs Tony outcomes
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Box plot: producer count by win status
shows_clean = shows.dropna(subset=['producer_count_total', 'tony_win_any'])
axes[0].boxplot([shows_clean[shows_clean['tony_win_any'] == 0]['producer_count_total'],
                 shows_clean[shows_clean['tony_win_any'] == 1]['producer_count_total']],
                labels=['No Win', 'Won Tony'])
axes[0].set_ylabel('Producer Count')
axes[0].set_title('Producer Count by Tony Win Status')

# Scatter: nominations vs producer count
axes[1].scatter(shows['producer_count_total'], shows['tony_nom_count'], alpha=0.6)
axes[1].set_xlabel('Producer Count')
axes[1].set_ylabel('Tony Nominations')
axes[1].set_title('Tony Nominations vs Producer Count')

# Add correlation
corr = shows[['producer_count_total', 'tony_nom_count']].corr().iloc[0, 1]
axes[1].text(0.05, 0.95, f'Correlation: {corr:.3f}', 
             transform=axes[1].transAxes, verticalalignment='top')

plt.tight_layout()
plt.savefig(config.OUTPUT_FIGURES_DIR / 'producer_count_vs_tony.png', dpi=300, bbox_inches='tight')
plt.show()

### Correlation Matrix

In [ ]:
# Correlation matrix of key variables
corr_vars = ['producer_count_total', 'lead_producer_count', 'co_producer_count',
             'tony_nom_count', 'tony_win_count', 'tony_win_any']
corr_matrix = shows[corr_vars].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix: Producers and Tony Outcomes')
plt.tight_layout()
plt.savefig(config.OUTPUT_FIGURES_DIR / 'correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nCorrelation Matrix:")
print(corr_matrix)

## 3. Logistic Regression: Tony Wins vs Producer Count

**Model:** Logit(P(Tony Win)) ~ producer_count + controls

**Outcome:** `tony_win_any` (1 if show won at least one Tony, 0 otherwise)

In [ ]:
# Prepare data for logistic regression
logit_data = shows[['tony_win_any', 'producer_count_total']].dropna()

# Add constant
X = sm.add_constant(logit_data['producer_count_total'])
y = logit_data['tony_win_any']

# Fit logistic regression
logit_model = sm.Logit(y, X).fit()

print("Logistic Regression: Tony Win ~ Producer Count")
print("=" * 60)
print(logit_model.summary())

In [ ]:
# Odds ratios
odds_ratios = np.exp(logit_model.params)
conf_int = np.exp(logit_model.conf_int())
conf_int.columns = ['2.5%', '97.5%']

odds_ratio_table = pd.DataFrame({
    'Odds Ratio': odds_ratios,
    '95% CI Lower': conf_int['2.5%'],
    '95% CI Upper': conf_int['97.5%'],
    'p-value': logit_model.pvalues
})

print("\nOdds Ratios:")
print(odds_ratio_table)

# Save to file
odds_ratio_table.to_csv(config.OUTPUT_TABLES_DIR / 'logit_tony_win_odds_ratios.csv')

In [ ]:
# Interpretation
producer_or = odds_ratios['producer_count_total']
pct_change = (producer_or - 1) * 100

print("\nInterpretation:")
print(f"Each additional producer is associated with a {pct_change:.1f}% change")
print(f"in the odds of winning at least one Tony Award.")

if logit_model.pvalues['producer_count_total'] < 0.05:
    print(f"This effect is statistically significant (p={logit_model.pvalues['producer_count_total']:.4f})")
else:
    print(f"This effect is NOT statistically significant (p={logit_model.pvalues['producer_count_total']:.4f})")

## 4. Count Model: Tony Nominations vs Producer Count

**Model:** Tony Nomination Count ~ producer_count + controls

Using Poisson regression (or Negative Binomial if over-dispersed)

In [ ]:
# Prepare data
count_data = shows[['tony_nom_count', 'producer_count_total']].dropna()

# Fit Poisson model
X_count = sm.add_constant(count_data['producer_count_total'])
y_count = count_data['tony_nom_count']

poisson_model = sm.Poisson(y_count, X_count).fit(cov_type='HC0')  # Robust SE

print("Poisson Regression: Tony Nominations ~ Producer Count")
print("=" * 60)
print(poisson_model.summary())

In [ ]:
# Test for over-dispersion
# If variance > mean, data is over-dispersed
print("\nOver-dispersion Test:")
print(f"Mean nominations: {y_count.mean():.3f}")
print(f"Variance: {y_count.var():.3f}")
print(f"Variance/Mean ratio: {y_count.var() / y_count.mean():.3f}")

if y_count.var() / y_count.mean() > 1.5:
    print("Data appears over-dispersed. Fitting Negative Binomial model...")
    
    # Use sm.NegativeBinomial instead of importing separately
    nb_model = sm.NegativeBinomial(y_count, X_count).fit()
    print("\nNegative Binomial Regression: Tony Nominations ~ Producer Count")
    print("=" * 60)
    print(nb_model.summary())
    
    # Use NB model for interpretation
    count_model = nb_model
else:
    print("Poisson model is appropriate.")
    count_model = poisson_model

In [ ]:
# Incidence rate ratios
irr = np.exp(count_model.params)
irr_conf = np.exp(count_model.conf_int())
irr_conf.columns = ['2.5%', '97.5%']

irr_table = pd.DataFrame({
    'IRR': irr,
    '95% CI Lower': irr_conf['2.5%'],
    '95% CI Upper': irr_conf['97.5%'],
    'p-value': count_model.pvalues
})

print("\nIncidence Rate Ratios:")
print(irr_table)

# Save to file
irr_table.to_csv(config.OUTPUT_TABLES_DIR / 'count_model_tony_nominations_irr.csv')

In [ ]:
# Interpretation
producer_irr = irr['producer_count_total']
pct_change_nom = (producer_irr - 1) * 100

print("\nInterpretation:")
print(f"Each additional producer is associated with a {pct_change_nom:.1f}% change")
print(f"in the expected number of Tony nominations.")

if count_model.pvalues['producer_count_total'] < 0.05:
    print(f"This effect is statistically significant (p={count_model.pvalues['producer_count_total']:.4f})")
else:
    print(f"This effect is NOT statistically significant (p={count_model.pvalues['producer_count_total']:.4f})")

## 5. Panel Model: Weekly Grosses vs Producer Count

**Model:** log(Weekly Gross) ~ producer_count + week + post_tony + controls

*(This section requires weekly grosses data)*

In [ ]:
if not weekly_grosses.empty:
    # Merge producer counts
    panel_data = weekly_grosses.merge(
        shows[['show_id', 'producer_count_total', 'lead_producer_count', 'co_producer_count']],
        on='show_id',
        how='left'
    )
    
    # Create log gross
    panel_data['log_weekly_gross'] = np.log(panel_data['weekly_gross'] + 1)
    
    # Drop missing
    panel_data = panel_data.dropna(subset=['log_weekly_gross', 'producer_count_total', 
                                            'week_number_since_open', 'post_tony_period'])
    
    print(f"Panel data: {len(panel_data)} observations, {panel_data['show_id'].nunique()} shows")
    
    # Fixed effects regression (within transformation)
    # Using statsmodels formula API
    panel_formula = 'log_weekly_gross ~ producer_count_total + week_number_since_open + post_tony_period'
    
    panel_model = smf.ols(panel_formula, data=panel_data).fit(
        cov_type='cluster', cov_kwds={'groups': panel_data['show_id']}
    )
    
    print("\nPanel Regression: Log(Weekly Gross) ~ Producer Count + Controls")
    print("=" * 60)
    print(panel_model.summary())
    
    # Save coefficients
    panel_coef = pd.DataFrame({
        'Coefficient': panel_model.params,
        'Std Error': panel_model.bse,
        't-statistic': panel_model.tvalues,
        'p-value': panel_model.pvalues,
        '95% CI Lower': panel_model.conf_int()[0],
        '95% CI Upper': panel_model.conf_int()[1]
    })
    panel_coef.to_csv(config.OUTPUT_TABLES_DIR / 'panel_regression_grosses.csv')
    
    # Interpretation
    producer_coef = panel_model.params['producer_count_total']
    pct_effect = (np.exp(producer_coef) - 1) * 100
    
    print("\nInterpretation:")
    print(f"Each additional producer is associated with a {pct_effect:.2f}% change")
    print(f"in weekly gross revenue, holding other factors constant.")
    
else:
    print("Weekly grosses data not available. Skipping panel regression.")
    print("TO DO: Obtain weekly grosses data to run this analysis.")

## 6. Survival Analysis: Run Length vs Producer Count

**Model:** Cox Proportional Hazards

**Outcome:** Time to closing (weeks)

*(This section requires opening/closing date data)*

In [ ]:
if not survival_panel.empty and 'duration' in survival_panel.columns:
    # Prepare data
    surv_data = survival_panel[['duration', 'event', 'producer_count_total']].dropna()
    
    print(f"Survival analysis: {len(surv_data)} shows")
    print(f"Events (closures): {surv_data['event'].sum()}")
    print(f"Censored (still running): {(1 - surv_data['event']).sum()}")
    
    # Kaplan-Meier curves by producer count quartile
    surv_data['producer_quartile'] = pd.qcut(surv_data['producer_count_total'], 
                                               q=4, labels=['Q1 (Low)', 'Q2', 'Q3', 'Q4 (High)'])
    
    kmf = KaplanMeierFitter()
    
    plt.figure(figsize=(12, 6))
    
    for quartile in surv_data['producer_quartile'].unique():
        mask = surv_data['producer_quartile'] == quartile
        kmf.fit(surv_data.loc[mask, 'duration'], 
                surv_data.loc[mask, 'event'],
                label=f'Producer Count {quartile}')
        kmf.plot_survival_function()
    
    plt.xlabel('Weeks Since Opening')
    plt.ylabel('Survival Probability (Show Still Running)')
    plt.title('Kaplan-Meier Survival Curves by Producer Count Quartile')
    plt.tight_layout()
    plt.savefig(config.OUTPUT_FIGURES_DIR / 'survival_curves_by_producer_count.png', 
                dpi=300, bbox_inches='tight')
    plt.show()
    
else:
    print("Survival data not available (missing duration/event columns).")
    print("TO DO: Obtain opening/closing dates to compute run length.")

In [ ]:
if not survival_panel.empty and 'duration' in survival_panel.columns:
    # Cox Proportional Hazards model
    cox_data = survival_panel[['duration', 'event', 'producer_count_total']].dropna()
    
    cph = CoxPHFitter()
    cph.fit(cox_data, duration_col='duration', event_col='event')
    
    print("\nCox Proportional Hazards Model")
    print("=" * 60)
    cph.print_summary()
    
    # Save coefficients
    cox_summary = cph.summary
    cox_summary.to_csv(config.OUTPUT_TABLES_DIR / 'cox_survival_model.csv')
    
    # Interpretation
    hr = cph.hazard_ratios_['producer_count_total']
    p_val = cph.summary.loc['producer_count_total', 'p']
    
    print("\nInterpretation:")
    print(f"Hazard Ratio for producer_count_total: {hr:.3f}")
    
    if hr < 1:
        pct_decrease = (1 - hr) * 100
        print(f"Each additional producer is associated with a {pct_decrease:.1f}% decrease")
        print(f"in the hazard of closing (i.e., longer run length).")
    else:
        pct_increase = (hr - 1) * 100
        print(f"Each additional producer is associated with a {pct_increase:.1f}% increase")
        print(f"in the hazard of closing (i.e., shorter run length).")
    
    if p_val < 0.05:
        print(f"This effect is statistically significant (p={p_val:.4f})")
    else:
        print(f"This effect is NOT statistically significant (p={p_val:.4f})")

## 7. Robustness Checks

### 7.1 Separate Effects: Lead vs Co-Producers

In [ ]:
# Logistic regression with separate producer types
logit_data_split = shows[['tony_win_any', 'lead_producer_count', 'co_producer_count']].dropna()

X_split = sm.add_constant(logit_data_split[['lead_producer_count', 'co_producer_count']])
y_split = logit_data_split['tony_win_any']

logit_split = sm.Logit(y_split, X_split).fit()

print("Logistic Regression: Tony Win ~ Lead Producers + Co-Producers")
print("=" * 60)
print(logit_split.summary())

# Odds ratios
odds_split = np.exp(logit_split.params)
print("\nOdds Ratios:")
print(odds_split)

### 7.2 Non-Linear Effects (Producer Count Quartiles)

In [ ]:
# Create producer count quartiles
shows['producer_quartile'] = pd.qcut(shows['producer_count_total'], 
                                       q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'], duplicates='drop')

# Logistic regression with quartile dummies
logit_quartile_data = shows[['tony_win_any', 'producer_quartile']].dropna()
logit_quartile_data = pd.get_dummies(logit_quartile_data, columns=['producer_quartile'], drop_first=True)

X_quartile = sm.add_constant(logit_quartile_data.drop(columns=['tony_win_any']))
y_quartile = logit_quartile_data['tony_win_any']

logit_quartile = sm.Logit(y_quartile, X_quartile).fit()

print("Logistic Regression: Tony Win ~ Producer Count Quartiles")
print("=" * 60)
print(logit_quartile.summary())

## 8. Summary of Findings

In [ ]:
print("="*80)
print("SUMMARY OF FINDINGS")
print("="*80)
print("\n1. DESCRIPTIVE STATISTICS")
print(f"   - Average producer count: {shows['producer_count_total'].mean():.1f}")
print(f"   - Shows with Tony wins: {shows['tony_win_any'].sum()} ({100*shows['tony_win_any'].mean():.1f}%)")
print(f"   - Average nominations: {shows['tony_nom_count'].mean():.1f}")

print("\n2. TONY WINS (Logistic Regression)")
producer_or = np.exp(logit_model.params['producer_count_total'])
print(f"   - Odds ratio per additional producer: {producer_or:.3f}")
print(f"   - P-value: {logit_model.pvalues['producer_count_total']:.4f}")
print(f"   - Significant: {logit_model.pvalues['producer_count_total'] < 0.05}")

print("\n3. TONY NOMINATIONS (Count Model)")
producer_irr = np.exp(count_model.params['producer_count_total'])
print(f"   - Incidence rate ratio: {producer_irr:.3f}")
print(f"   - P-value: {count_model.pvalues['producer_count_total']:.4f}")
print(f"   - Significant: {count_model.pvalues['producer_count_total'] < 0.05}")

if not weekly_grosses.empty:
    print("\n4. WEEKLY GROSSES (Panel Regression)")
    print(f"   - Coefficient: {panel_model.params['producer_count_total']:.4f}")
    print(f"   - P-value: {panel_model.pvalues['producer_count_total']:.4f}")
else:
    print("\n4. WEEKLY GROSSES: Data not available")

if not survival_panel.empty and 'duration' in survival_panel.columns:
    print("\n5. SURVIVAL ANALYSIS (Cox PH)")
    print(f"   - Hazard ratio: {hr:.3f}")
    print(f"   - P-value: {p_val:.4f}")
else:
    print("\n5. SURVIVAL ANALYSIS: Data not available")

print("\n" + "="*80)
print("Analysis complete. Results saved to output/ directory.")
print("="*80)

## 9. Limitations and Future Work

### Current Limitations:
1. **Missing opening/closing dates**: Limits survival analysis and run length calculations
2. **Missing review scores**: Cannot control for critical reception
3. **Missing theatre metadata**: Cannot control for venue size, location
4. **Limited grosses data**: May not have complete coverage of all shows
5. **No capitalization data**: Cannot control for production budget
6. **No star power indicator**: Cannot control for celebrity casting

### Future Work:
1. Scrape IBDB for complete show metadata (dates, theatre, category)
2. Add NYT review sentiment analysis (manual coding or NLP)
3. Expand grosses data collection
4. Add interaction effects (e.g., producer count × musical/play)
5. Investigate specific producer networks and repeat collaborations
6. Analyze temporal trends: has producer count increased over time?

### Data Quality Notes:
- Producer counts may be incomplete for some shows (scraping limitations)
- Title matching between data sources may have errors (fuzzy matching used)
- Tony data scraped from Wikipedia (verify against official TonyAwards.com)

**All analyses subject to data availability and quality constraints.**